In [1]:
pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00


In [2]:
pip install geopy requests

In [3]:
import gradio as gr
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from geopy.geocoders import Nominatim
import requests
import random
import time

# -------- Sleep Quality Predictor --------
def predict_sleep_quality(sleep_duration, heart_rate, activity_level):
    df = pd.read_csv("Sleep_health_and_lifestyle_dataset.csv")
    df.dropna(subset=['Sleep Duration', 'Heart Rate', 'Physical Activity Level', 'Quality of Sleep'], inplace=True)

    y = df['Quality of Sleep']
    X = df[['Sleep Duration', 'Heart Rate', 'Physical Activity Level']]
    model = RandomForestClassifier()
    model.fit(X, y)

    prediction = model.predict([[sleep_duration, heart_rate, activity_level]])[0]

    advice = ""
    if sleep_duration < 5:
        advice += "⚠ Your sleep duration is low. Aim for 7–9 hours.\n"
    if heart_rate > 100:
        advice += "❤ High heart rate. Consider stress reduction or hydration.\n"
    elif heart_rate < 50:
        advice += "💤 Low heart rate. Monitor for fatigue or consult a doctor.\n"
    if activity_level < 30:
        advice += "🏃 Low physical activity. Try to be more active daily.\n"

    return f"🌙 *Predicted Sleep Quality:* {prediction}\n\n{advice or '✅ All inputs are within healthy range!'}"

# -------- Disease Predictor --------
def predict_disease(symptoms):
    if not symptoms:
        return "⚠ Please enter symptoms.", ""

    user_symptoms = [s.strip().lower() for s in symptoms.split(',')]

    # Loading Dataset 1
    try:
        df1 = pd.read_csv('Disease_symptom_and_patient_profile_dataset.csv', encoding='latin1')
    except Exception as e:
        return f"⚠ Dataset 1 Error: {str(e)}", ""

    disease_col1 = 'Disease'
    symptom_cols1 = [col for col in df1.columns if col.lower() != disease_col1.lower()]
    df1_symptoms = [col.lower() for col in symptom_cols1]

    symptom_mask1 = pd.Series(False, index=df1.index)
    for symptom in user_symptoms:
        if symptom in df1_symptoms:
            matched_col = symptom_cols1[df1_symptoms.index(symptom)]
            symptom_mask1 |= (df1[matched_col].astype(str).str.lower() == 'yes')

    matching_diseases1 = df1[symptom_mask1][disease_col1].unique()

    # Loading  Dataset 2
    try:
        df2 = pd.read_csv('Testing.csv', encoding='latin1')
    except Exception as e:
        return f"⚠ Dataset 2 Error: {str(e)}", ""

    df2.fillna(0, inplace=True)
    X = df2.drop(columns=['prognosis'])
    y = df2['prognosis']

    symptom_cols2 = X.columns.str.lower().tolist()
    input_vector = [1 if col in user_symptoms else 0 for col in symptom_cols2]

    model = RandomForestClassifier()
    model.fit(X, y)

    probs = model.predict_proba([input_vector])[0]
    top_indices = probs.argsort()[-3:][::-1]
    top_preds = [model.classes_[i] for i in top_indices]

    # clean version of the output
    output = "### 🩺 Disease Risk Assessment\n"
    for dis in matching_diseases1[:3]:
      output += f"- *{dis}* (if any previous medical history)\n"

    for dis in top_preds:
        output += f"- *{dis}*\n"
    output += "\n*Note: This is not a medical diagnosis. Consult a doctor for real concerns.*"

    return output, top_preds[0]




# -------- Doctor Finder --------
def find_doctors(disease, location):
    if not disease or not location:
        return "⚠ Please provide both disease and location."

    try:
        geolocator = Nominatim(user_agent="simple_health_app")
        loc = geolocator.geocode(location)

        if not loc:
            return "⚠ Unable to find that location. Please try another city or area."

        query = "doctor"
        search_params = {
            'q': query,
            'format': 'json',
            'limit': 5,
            'viewbox': f"{loc.longitude - 0.1},{loc.latitude + 0.1},{loc.longitude + 0.1},{loc.latitude - 0.1}",
            'bounded': 1
        }
        headers = {'User-Agent': 'simple_health_app'}
        response = requests.get("https://nominatim.openstreetmap.org/search", params=search_params, headers=headers)

        if response.status_code == 503:
            time.sleep(1)
            response = requests.get("https://nominatim.openstreetmap.org/search", params=search_params, headers=headers)

        if response.status_code != 200:
            return f"⚠ Failed to get data. Status code: {response.status_code}"

        results = response.json()
        if not results:
            return "⚠ No doctors found nearby. Try a different or larger location."

        result_text = "### 🧑‍⚕️ Nearby Doctors:\n"
        for item in results:
            name = item.get('display_name', 'Unknown')
            result_text += f"- {name}\n"

        return result_text

    except Exception as e:
        return f"⚠ Error: {str(e)}"

# -------- Nutrition Recommender --------
def nutrition_recommendation(calories, carbs, protein, fat):
    nutri_df = pd.read_csv("nutrition.csv")
    for col in ['calories', 'carbohydrate', 'protein', 'fat']:
        nutri_df[col] = nutri_df[col].astype(str).str.replace('g', '').str.strip()
        nutri_df[col] = pd.to_numeric(nutri_df[col], errors='coerce')

    nutri_df.dropna(subset=['calories', 'carbohydrate', 'protein', 'fat'], inplace=True)
    if nutri_df.empty:
        return "⚠ No valid food data available."

    feat = nutri_df[['calories', 'carbohydrate', 'protein', 'fat']]
    scaler_n = StandardScaler().fit(feat)
    scaled_feat = scaler_n.transform(feat)
    knn = NearestNeighbors(n_neighbors=10).fit(scaled_feat)

    user_input = [[calories, carbs, protein, fat]]
    scaled_user_input = scaler_n.transform(user_input)

    distances, indices = knn.kneighbors(scaled_user_input)
    result = "### 🍽 Top 10 Food Recommendations:\n"

    for i, idx in enumerate(indices[0]):
        row = nutri_df.iloc[idx]
        result += f"\n*{i + 1}. {row['name']}*\n"
        result += f"   - Calories: {row['calories']}\n"
        result += f"   - Carbs: {row['carbohydrate']}g\n"
        result += f"   - Protein: {row['protein']}g\n"
        result += f"   - Fat: {row['fat']}g\n"

    return result

# -------- Gradio UI --------
def create_ui():
    custom_css = """
    #bg-animation {
        position: fixed;
        top: 0;
        left: 0;
        width: 100vw;
        height: 100vh;
        object-fit: cover;
        z-index: -1;
        opacity: 0.15;
    }

    .gr-block {
        animation: fadein 0.6s ease-in-out;
    }
    @keyframes fadein {
        from { opacity: 0; transform: translateY(12px); }
        to { opacity: 1; transform: translateY(0); }
    }

    .gr-button {
        transition: all 0.3s ease;
        background: linear-gradient(135deg, #4CAF50, #2E7D32);
        color: white;
        border-radius: 10px;
        padding: 10px 16px;
        font-weight: bold;
        box-shadow: 0 4px 6px rgba(0,0,0,0.1);
    }
    .gr-button:hover {
        transform: scale(1.05);
        background: linear-gradient(135deg, #66BB6A, #388E3C);
        box-shadow: 0 6px 10px rgba(0,0,0,0.15);
    }

    .gr-markdown {
        background-color: #ffffff;
        padding: 16px;
        border-radius: 16px;
        box-shadow: 0 2px 8px rgba(0,0,0,0.05);
        transition: transform 0.3s ease, box-shadow 0.3s ease;
    }
    .gr-markdown:hover {
        transform: scale(1.01);
        box-shadow: 0 4px 16px rgba(0,0,0,0.08);
    }

    .gr-textbox:hover, .gr-number:hover, .gr-slider:hover {
        box-shadow: 0 0 6px rgba(76, 175, 80, 0.5);
        transition: box-shadow 0.2s ease;
        border-radius: 10px !important;
    }

    #header {
        font-size: 30px;
        font-weight: bold;
        color: #388E3C;
        text-align: center;
        padding-bottom: 10px;
    }
    """

    with gr.Blocks(css=custom_css) as demo:
        gr.Image(
            value="https://media.giphy.com/media/3o7abB06u9bNzA8lu8/giphy.gif",  # replace with your desired gif
            show_label=False,
            interactive=False,
            show_download_button=False,
            elem_id="bg-animation"
        )

        gr.Markdown("## 🧠 <span style='color:#388E3C'>AI Medical Assistant</span>", elem_id="header")
        gr.Markdown("Welcome! Select a health tool below. All features are 💯 free and private.")

        with gr.Tabs():
            with gr.Tab("🛌 Sleep Quality Predictor"):
                with gr.Row():
                    with gr.Column():
                        sd = gr.Slider(0, 12, label="🕒 Sleep Duration (hours)", interactive=True)
                        hr = gr.Number(label="❤️ Heart Rate (bpm)", interactive=True)
                        act = gr.Slider(0, 500, step=10, label="🏃 Activity Level (minutes)", interactive=True)
                        btn1 = gr.Button("🧪 Predict Sleep Quality")
                    with gr.Column():
                        sleep_out = gr.Markdown("### ⏳ Sleep Prediction Will Appear Here")
                btn1.click(predict_sleep_quality, [sd, hr, act], sleep_out)

            with gr.Tab("🩺 Disease Risk Estimator"):
                with gr.Row():
                    with gr.Column():
                        symptoms = gr.Textbox(label="💬 Symptoms (comma-separated)")
                        location = gr.Textbox(label="🌍 Your Location (e.g., Delhi, NYC)")
                        btn2 = gr.Button("🧬 Analyze Risk")
                        predicted_disease = gr.State()
                        disease_out = gr.Markdown("### 🧪 Disease Risk Will Appear Here")
                        btn2.click(predict_disease, [symptoms], [disease_out, predicted_disease])

                        btn3 = gr.Button("🧑‍⚕️ Find Nearby Doctors")
                        doctor_out = gr.Markdown("### 🏥 Doctor Info Will Appear Here")
                        btn3.click(find_doctors, [predicted_disease, location], doctor_out)

            with gr.Tab("🥗 Nutrition Recommender"):
                with gr.Row():
                    with gr.Column():
                        cal = gr.Number(label="🔥 Target Calories")
                        carb = gr.Number(label="🥔 Carbohydrates (g)")
                        prot = gr.Number(label="🍗 Protein (g)")
                        fat = gr.Number(label="🧈 Fat (g)")
                        btn4 = gr.Button("🍽 Recommend Foods")
                    with gr.Column():
                        nutr_out = gr.Markdown("### 🧾 Food Suggestions Will Appear Here")
                btn4.click(nutrition_recommendation, [cal, carb, prot, fat], nutr_out)

        gr.Markdown("---")
        gr.Markdown("<div style='text-align: center;'>Made with ❤️ using <b>Gradio</b> and <b>Python</b></div>")

    return demo

app = create_ui()
app.launch(share=True, inline=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e62a330bba5c9f6472.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
